네이버 영화 댓글 감성분석
http://github.com/e9t/nsmc

In [2]:
!uv pip install requests pandas scikit-learn kiwipiepy

Using Python 3.12.10 environment at: /Users/jiyouxg/Desktop/SKN21_mjy/08_NLP-자연어처리/.venv
Audited 4 packages in 16ms


In [16]:
import requests
import pandas as pd
from io import StringIO

url = "https://raw.githubusercontent.com/e9t/nsmc/refs/heads/master/ratings.txt"
res = requests.get(url)
if res.status_code == 200:
    str_io = StringIO(res.text)  # 메모리의 문자열 (str)로부터 값을 읽을 수 있는 InputStream
    df = pd.read_csv(str_io, sep="\t")



In [6]:
df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        200000 non-null  int64 
 1   document  199992 non-null  object
 2   label     200000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [17]:
df.isnull().sum()

id          0
document    8
label       0
dtype: int64

In [18]:
# 결측치 제거
df = df.dropna()
df.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [19]:
# 중복된 댓글 (document) 확인 (중복행 확인)
df[df.duplicated(subset=["document"], keep=False)].sort_values('document')
# keep: False - 모든 중복행들을 True, "first", "last" (중복된 것들 중 첫 번째/마지막 것만 True)

,id,document,label
136580,6993402,!,0
12986,181912,!,1
59493,7448690,",",1
15611,7868198,",",1
101742,7481337,",,,",0
...,...,...,...
1961,5158304,힐러리 더프의 매력에 빠지다!!!,1
104378,3010576,힘내세요,0
152847,4052413,힘내세요,0
118102,7024515,힘들다,0


In [20]:
# 중복된 댓글이 있는 행은 하나만 남기고 제거
df.drop_duplicates(subset='document')

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
...,...,...,...
199995,8963373,포켓 몬스터 짜가 ㅡㅡ;;,0
199996,3302770,쓰.레.기,0
199997,5458175,완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.,0
199998,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0


In [ ]:
df[df.duplicated]

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
# 네이버 댓글 토큰화 처리 클래스 
# 1. 다운로드
# 2. 기본적인 전처리 진행
# 3. 형태소 기반 토큰화 진행 
import os 
import requests
import pandas as pd
from kiwipiepy import Kiwi

class NSMCTokenizer:

    def __init__(self, save_path:str):
        # save_path: 데이터셋 다운 후 저장할 파일 경로
        self.kiwi = Kiwi(num_workers=-1)  # num_works: 병렬 처리 시 사용할 cpu 개수. -1: 모두

        # nsmc 데이터셋을 loading - load_nsmc_dataset(save_path)
        df = self.load_nsmc_dataset(save_path)

    def load_nsmc_dataset(self, save_path:str="data")->pd.DataFrame:
        """
        NSMC 데이터셋을 다운 받아 저장 및 DataFrame으로 반환.
        Args:
            save_path: 데이터셋 csv 파일을 저장할 디렉토리
        Returns:
            pd.DataFrame
                네이버 영화 댓글 감성 분석 데이터셋.
                feature: id-댓글 ID, document: 댓글 내용, label: Target(0:부정, 1:긍정)
        """
        # 디렉토리 생성
        os.makedirs(save_path, exist_ok=True)
        file_path = os.path.join(save_path, 'ratings.txt')
        
        try:
            df = pd.read_csv(file_path, sep='\t', encoding='UTF-8')
        except:
            # Exception 발생: csv파일이 없거나 잘못 저장된 경우 -> 다운로드
            if os.path.exists(file_path):   # True: 있는 파일/디렉토리, False: 없다. 
                os.remove(file_path)
            url = "https://raw.githubusercontent.com/e9t/nsmc/refs/heads/master/ratings.txt"
            res = requests.get(url)
            if res.status_code == 200:
                # file_path에 저장
                with open(file_path, 'wt', encoding="utf-8") as fw:
                    fw.write(res.text)
                df=pd.read_csv(file_path, sep='\t', encoding="utf-8")
            else:
                # 다운 시 문제 발생
                raise Exception("csv 파일을 다운받지 못했습니다. status코드:", res.status_code)   
                     
        return df

    def tokenize(self, doc:str)->str:
        """
        개별 댓글을 받아서 공백 처리, 토큰화 처리를 한 결과를 다시 string으로 만들어서 반환.

        Args:
            doc:str - 처리할 댓글 문서 1개
        
        Returns:
            str - 처리 결과
        """

        doc = self.kiwi.space(doc)
        token_list = []
        try:
            # 토큰화: 원형 (lemma)를 저장.
            self.kiwi.tokenize(doc)
            token_list.append(token.lemma)
        except:
            pass

    def preprocess(self, df:pd.DataFrame)->pd.DataFrame:
        """
        Dataset 전처리 + 토큰화 작업
        전처리: 결측치 제거, 댓글 중복 데이터 (중복행) 삭제
        토큰화: tokenize() 메소드를 이용해 토큰화 작업
        Args: 
            df:pd.DataFrame - 전처리 대상 DataFrame
            
        Returns:
            pd.DataFrame - 전처리 결과
        """
        res_df = df.dropna() # 결측치 처리
        res_df = res_df.drop_duplicates(subset="document") # 중복행 제거

        # 공백 교정, 토큰화 -> tokenize()
        res_df['document'] = res_df['document'].apply(self.tokenize)

        # 공백 제거 후 글자 수가 0인 행의 index 조회
        drop_idx = res_df[res_df['document'].str.strip().str.len()==0].index
        res_df.drop(index=drop_idx).reset_index(drop=True)
        return res_df

In [25]:
import time
s=time.time()

nsmc = NSMCTokenizer("data")

print(time.time() - s, "초")

1.3872919082641602 초


In [26]:
# 확인
nsmc.nsmc_df.shape

AttributeError: 'NSMCTokenizer' object has no attribute 'nsmc_df'

In [ ]:
nsmc.nsmc_df.head(10)

In [15]:
# 문자열 타입 Series.str.메소드 -> str accessor: Series의 원소들을 문자열 관련 처리를 일괄 처리. 
# accessor: str(문자열 처리), dt(일시-datatime 타입 시리즈), plot: 시각화.